필요한 라이브러리 불러오기


In [359]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor

from xgboost.sklearn import XGBClassifier,XGBRegressor
from lightgbm.sklearn import LGBMClassifier,LGBMRegressor

from sklearn.model_selection import  KFold
from functools import partial

import optuna


데이터 불러오기

In [394]:
# data 폴더에 있는 모든 csv 파일을 읽어오기 위해 glob을 사용합니다.
from glob import glob

file_paths = sorted(glob('C:/Users/jeong bok/Documents/cse101/team9PJ/mlops-team9/data/*.xlsx', key=lambda x: int(''.join(filter(str.isdigit, x)))))
# csv 목록 불러오기
total = pd.DataFrame()
for file_path in file_paths:
    temp = pd.read_excel(file_path,sheet_name=1)
    total = pd.concat([total,temp])


# 모든 csv 병합하기
total

,일자,요일,지점명,지점번호,방향,구분,0시,1시,2시,3시,...,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시
0,20181001,월,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,419.0,308.0,227.0,191.0,...,1925.0,2012.0,2104.0,2301.0,2246.0,1743.0,1400.0,1508.0,1331.0,983.0
1,20181002,화,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,669.0,508.0,324.0,283.0,...,1977.0,2128.0,2221.0,2394.0,2306.0,1961.0,1685.0,1597.0,1536.0,1013.0
2,20181003,수,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,785.0,547.0,479.0,396.0,...,1463.0,1602.0,1719.0,1768.0,1640.0,1405.0,1292.0,1247.0,1007.0,721.0
3,20181004,목,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,507.0,323.0,198.0,231.0,...,1970.0,2050.0,2311.0,2451.0,2198.0,1796.0,1527.0,1557.0,1457.0,991.0
4,20181005,금,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,701.0,595.0,389.0,337.0,...,2076.0,2029.0,2070.0,2137.0,2157.0,1998.0,1406.0,1392.0,1236.0,977.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8095,20180926,수,서부간선도로,F-09,유출,목동교->신정교,2338.0,1790.0,1412.0,1147.0,...,2137.0,3135.0,3001.0,3016.0,2875.0,3085.0,3158.0,3168.0,3062.0,2515.0
8096,20180927,목,서부간선도로,F-09,유출,목동교->신정교,1893.0,1269.0,910.0,842.0,...,3122.0,3086.0,2925.0,2901.0,2797.0,3014.0,3214.0,3414.0,3280.0,3075.0
8097,20180928,금,서부간선도로,F-09,유출,목동교->신정교,2414.0,1715.0,1253.0,959.0,...,3123.0,2994.0,2894.0,3004.0,2684.0,2827.0,3092.0,3249.0,3123.0,3316.0
8098,20180929,토,서부간선도로,F-09,유출,목동교->신정교,3014.0,2640.0,2066.0,1785.0,...,2962.0,2971.0,2855.0,2868.0,2829.0,2865.0,2965.0,3122.0,3006.0,3131.0


In [395]:
total.head(5)


,일자,요일,지점명,지점번호,방향,구분,0시,1시,2시,3시,...,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시
0,20181001,월,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,419.0,308.0,227.0,191.0,...,1925.0,2012.0,2104.0,2301.0,2246.0,1743.0,1400.0,1508.0,1331.0,983.0
1,20181002,화,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,669.0,508.0,324.0,283.0,...,1977.0,2128.0,2221.0,2394.0,2306.0,1961.0,1685.0,1597.0,1536.0,1013.0
2,20181003,수,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,785.0,547.0,479.0,396.0,...,1463.0,1602.0,1719.0,1768.0,1640.0,1405.0,1292.0,1247.0,1007.0,721.0
3,20181004,목,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,507.0,323.0,198.0,231.0,...,1970.0,2050.0,2311.0,2451.0,2198.0,1796.0,1527.0,1557.0,1457.0,991.0
4,20181005,금,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,701.0,595.0,389.0,337.0,...,2076.0,2029.0,2070.0,2137.0,2157.0,1998.0,1406.0,1392.0,1236.0,977.0


In [397]:


tr_df = total
air_df = pd.read_csv('C:/Users/jeong bok/Documents/cse101/team9PJ/mlops-team9/data/new_AIR_HOUR_2018.csv')

gu_df = pd.read_csv('C:/Users/jeong bok/Documents/cse101/team9PJ/mlops-team9/data/spot_data_gu.csv')

In [398]:
tr_df

,일자,요일,지점명,지점번호,방향,구분,0시,1시,2시,3시,...,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시
0,20181001,월,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,419.0,308.0,227.0,191.0,...,1925.0,2012.0,2104.0,2301.0,2246.0,1743.0,1400.0,1508.0,1331.0,983.0
1,20181002,화,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,669.0,508.0,324.0,283.0,...,1977.0,2128.0,2221.0,2394.0,2306.0,1961.0,1685.0,1597.0,1536.0,1013.0
2,20181003,수,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,785.0,547.0,479.0,396.0,...,1463.0,1602.0,1719.0,1768.0,1640.0,1405.0,1292.0,1247.0,1007.0,721.0
3,20181004,목,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,507.0,323.0,198.0,231.0,...,1970.0,2050.0,2311.0,2451.0,2198.0,1796.0,1527.0,1557.0,1457.0,991.0
4,20181005,금,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,701.0,595.0,389.0,337.0,...,2076.0,2029.0,2070.0,2137.0,2157.0,1998.0,1406.0,1392.0,1236.0,977.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8095,20180926,수,서부간선도로,F-09,유출,목동교->신정교,2338.0,1790.0,1412.0,1147.0,...,2137.0,3135.0,3001.0,3016.0,2875.0,3085.0,3158.0,3168.0,3062.0,2515.0
8096,20180927,목,서부간선도로,F-09,유출,목동교->신정교,1893.0,1269.0,910.0,842.0,...,3122.0,3086.0,2925.0,2901.0,2797.0,3014.0,3214.0,3414.0,3280.0,3075.0
8097,20180928,금,서부간선도로,F-09,유출,목동교->신정교,2414.0,1715.0,1253.0,959.0,...,3123.0,2994.0,2894.0,3004.0,2684.0,2827.0,3092.0,3249.0,3123.0,3316.0
8098,20180929,토,서부간선도로,F-09,유출,목동교->신정교,3014.0,2640.0,2066.0,1785.0,...,2962.0,2971.0,2855.0,2868.0,2829.0,2865.0,2965.0,3122.0,3006.0,3131.0


In [415]:
tr_df.nunique()

일자       365
요일         7
지점명      190
지점번호     135
방향       324
구분       271
0시      4222
1시      3260
2시      2653
3시      2231
4시      2558
5시      4752
6시      6569
7시      6834
8시      6770
9시      6679
10시     6737
11시     6784
12시     6736
13시     6864
14시     6848
15시     6858
16시     6819
17시     6615
18시     6422
19시     6232
20시     6261
21시     6466
22시     6278
23시     5344
dtype: int64

In [400]:
tr_df.fillna(0,inplace=True)

In [401]:
air_df.head(5)

,Unnamed: 0,날짜,시간,구,PM10,PM2.5,O3,SO2,NO2,CO
0,0,2018-01-01,0,종로구,29.0,17.0,0.022,0.006,0.021,0.4
1,1,2018-01-01,0,중구,34.0,19.0,0.020,0.004,0.020,0.5
2,2,2018-01-01,0,용산구,29.0,14.0,0.013,0.004,0.023,0.3
3,3,2018-01-01,0,은평구,43.0,25.0,0.017,0.003,0.018,0.6
4,4,2018-01-01,0,서대문구,0.0,0.0,0.000,0.000,0.000,0.0


In [402]:
gu_df.head(5)

,spot_num,spot_nm,gu
0,A-01,성산로(금화터널),서대문구
1,A-02,사직로(사직터널),종로구
2,A-03,자하문로(자하문터널),종로구
3,A-04,대사관로(삼청터널),종로구
4,A-05,율곡로(안국역),종로구


데이터 전처리를 진행합니다

In [403]:
#칼럼값으로 퍼져있는 시간을 로우로 내림
melted_df = pd.melt(tr_df, id_vars=['일자', '요일', '지점명', '지점번호', '구분', '방향'],
                    var_name='시간', value_name='값')

In [404]:
melted_df.head(5)

,일자,요일,지점명,지점번호,구분,방향,시간,값
0,20181001,월,성산로(금화터널),A-01,봉원고가차도->독립문역,유입,0시,419.0
1,20181002,화,성산로(금화터널),A-01,봉원고가차도->독립문역,유입,0시,669.0
2,20181003,수,성산로(금화터널),A-01,봉원고가차도->독립문역,유입,0시,785.0
3,20181004,목,성산로(금화터널),A-01,봉원고가차도->독립문역,유입,0시,507.0
4,20181005,금,성산로(금화터널),A-01,봉원고가차도->독립문역,유입,0시,701.0


In [405]:
#결측치 확인 후 채움
melted_df.isna().value_counts()

melted_df.fillna('0',inplace=True)

In [406]:
melted_df.isna().value_counts()

일자     요일     지점명    지점번호   구분     방향     시간     값    
False  False  False  False  False  False  False  False    2365200
Name: count, dtype: int64

칼럼명 영어로 변경

In [407]:
new_column_names = {
    '일자': 'date',
    '시간': 'time',
    '지점번호': 'spot_num',
    '지점명': 'spot_nm',
    '요일' : 'day' ,
    '값' : 'traffic',
    '날짜' : 'date',
    '구'   : 'gu'
}

melted_df.rename(columns=new_column_names, inplace=True)
air_df.rename(columns=new_column_names, inplace=True)

In [408]:
# 필요없다고 생각된 구분 칼럼 삭제
melted_df = melted_df.drop(melted_df.columns[4], axis=1)

In [409]:
melted_df

,date,day,spot_nm,spot_num,방향,time,traffic
0,20181001,월,성산로(금화터널),A-01,유입,0시,419.0
1,20181002,화,성산로(금화터널),A-01,유입,0시,669.0
2,20181003,수,성산로(금화터널),A-01,유입,0시,785.0
3,20181004,목,성산로(금화터널),A-01,유입,0시,507.0
4,20181005,금,성산로(금화터널),A-01,유입,0시,701.0
...,...,...,...,...,...,...,...
2365195,20180926,수,서부간선도로,F-09,유출,23시,2515.0
2365196,20180927,목,서부간선도로,F-09,유출,23시,3075.0
2365197,20180928,금,서부간선도로,F-09,유출,23시,3316.0
2365198,20180929,토,서부간선도로,F-09,유출,23시,3131.0


In [410]:

air_df = air_df.drop(air_df.columns[0], axis=1)

In [413]:
melted_df.nunique()

date         365
day            7
spot_nm      190
spot_num     135
방향           324
time          24
traffic     8102
dtype: int64

In [412]:
#데이트타임 적용
melted_df['time']=melted_df['time'].str.replace('시','')
melted_df['datetime'] = pd.to_datetime(melted_df['date'].astype(str) + ' ' + melted_df['time'], format='%Y%m%d %H')
melted_df['datetime'] = melted_df['datetime'].dt.floor('H')
melted_df.drop(['date', 'time'], axis=1, inplace=True)



MemoryError: Unable to allocate 18.0 MiB for an array with shape (2365200,) and data type uint64

In [ ]:
traffic_df=melted_df

In [ ]:
# 'time' 열의 값이 단일 자릿수인 경우, 두 자리로 맞추기
air_df['time'] = air_df['time'].astype(int)
air_df['time'].dtype
#df_air['datetime']=df_air[df_air['date']+df_air['time']]


dtype('int32')

In [ ]:
# 'time' 열을 문자열로 변환하고, 두 자리로 만듭니다.
air_df['time'] = air_df['time'].astype(str).str.zfill(2)

# 'datetime' 열 생성
air_df['datetime'] = pd.to_datetime(air_df['date'].astype(str) + ' ' + air_df['time'], format='%Y-%m-%d %H', errors='coerce')

# 결과 확인
print(air_df.head())


         date time    gu  PM10  PM2.5     O3    SO2    NO2   CO   datetime
0  2018-01-01   00   종로구  29.0   17.0  0.022  0.006  0.021  0.4 2018-01-01
1  2018-01-01   00    중구  34.0   19.0  0.020  0.004  0.020  0.5 2018-01-01
2  2018-01-01   00   용산구  29.0   14.0  0.013  0.004  0.023  0.3 2018-01-01
3  2018-01-01   00   은평구  43.0   25.0  0.017  0.003  0.018  0.6 2018-01-01
4  2018-01-01   00  서대문구   0.0    0.0  0.000  0.000  0.000  0.0 2018-01-01


In [ ]:
# 'time' 열을 문자열로 변환
air_df['time'] = air_df['time'].astype(str)

# 'time' 열의 고유값 확인
print(air_df['time'].unique())


['00' '01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '11' '12' '13'
 '14' '15' '16' '17' '18' '19' '20' '21' '22' '23']


In [ ]:
dr_col = [
    'date','time'
]
air_df.drop(columns=dr_col)

,gu,PM10,PM2.5,O3,SO2,NO2,CO,datetime
0,종로구,29.0,17.0,0.022,0.006,0.021,0.4,2018-01-01 00:00:00
1,중구,34.0,19.0,0.020,0.004,0.020,0.5,2018-01-01 00:00:00
2,용산구,29.0,14.0,0.013,0.004,0.023,0.3,2018-01-01 00:00:00
3,은평구,43.0,25.0,0.017,0.003,0.018,0.6,2018-01-01 00:00:00
4,서대문구,0.0,0.0,0.000,0.000,0.000,0.0,2018-01-01 00:00:00
...,...,...,...,...,...,...,...,...
264746,관악구,36.0,24.0,0.004,0.005,0.059,0.6,2018-12-31 23:00:00
264747,서초구,36.0,13.0,0.002,0.004,0.052,0.5,2018-12-31 23:00:00
264748,강남구,35.0,26.0,0.002,0.005,0.044,0.7,2018-12-31 23:00:00
264749,송파구,43.0,27.0,0.002,0.004,0.057,0.9,2018-12-31 23:00:00


In [ ]:
melted_df

,day,spot_nm,spot_num,방향,traffic,datetime
0,월,성산로(금화터널),A-01,유입,657.0,2018-01-01 00:00:00
1,화,성산로(금화터널),A-01,유입,382.0,2018-01-02 00:00:00
2,수,성산로(금화터널),A-01,유입,641.0,2018-01-03 00:00:00
3,목,성산로(금화터널),A-01,유입,661.0,2018-01-04 00:00:00
4,금,성산로(금화터널),A-01,유입,764.0,2018-01-05 00:00:00
...,...,...,...,...,...,...
200875,토,서부간선도로,F-09,유출,2736.0,2018-01-27 23:00:00
200876,일,서부간선도로,F-09,유출,2278.0,2018-01-28 23:00:00
200877,월,서부간선도로,F-09,유출,2493.0,2018-01-29 23:00:00
200878,화,서부간선도로,F-09,유출,2185.0,2018-01-30 23:00:00


In [ ]:
merged_df = pd.merge(traffic_df, gu_df, how='left', left_on='spot_num', right_on='spot_num')

In [ ]:
merged_df

,day,spot_nm_x,spot_num,방향,traffic,datetime,spot_nm_y,gu
0,월,성산로(금화터널),A-01,유입,657.0,2018-01-01 00:00:00,성산로(금화터널),서대문구
1,화,성산로(금화터널),A-01,유입,382.0,2018-01-02 00:00:00,성산로(금화터널),서대문구
2,수,성산로(금화터널),A-01,유입,641.0,2018-01-03 00:00:00,성산로(금화터널),서대문구
3,목,성산로(금화터널),A-01,유입,661.0,2018-01-04 00:00:00,성산로(금화터널),서대문구
4,금,성산로(금화터널),A-01,유입,764.0,2018-01-05 00:00:00,성산로(금화터널),서대문구
...,...,...,...,...,...,...,...,...
200875,토,서부간선도로,F-09,유출,2736.0,2018-01-27 23:00:00,서부간선지하도로,영등포구
200876,일,서부간선도로,F-09,유출,2278.0,2018-01-28 23:00:00,서부간선지하도로,영등포구
200877,월,서부간선도로,F-09,유출,2493.0,2018-01-29 23:00:00,서부간선지하도로,영등포구
200878,화,서부간선도로,F-09,유출,2185.0,2018-01-30 23:00:00,서부간선지하도로,영등포구


In [ ]:
drop_col = ['spot_nm_x','spot_num','spot_nm_y']
merged_df.drop(columns=drop_col)

,day,방향,traffic,datetime,gu
0,월,유입,657.0,2018-01-01 00:00:00,서대문구
1,화,유입,382.0,2018-01-02 00:00:00,서대문구
2,수,유입,641.0,2018-01-03 00:00:00,서대문구
3,목,유입,661.0,2018-01-04 00:00:00,서대문구
4,금,유입,764.0,2018-01-05 00:00:00,서대문구
...,...,...,...,...,...
200875,토,유출,2736.0,2018-01-27 23:00:00,영등포구
200876,일,유출,2278.0,2018-01-28 23:00:00,영등포구
200877,월,유출,2493.0,2018-01-29 23:00:00,영등포구
200878,화,유출,2185.0,2018-01-30 23:00:00,영등포구


In [ ]:
# 방향을 기준으로 groupby하고, traffic을 합산
df_combined = merged_df.groupby(['day', 'datetime', 'gu'])['traffic'].sum().reset_index()

# 방향 column은 필요 없으므로 제거
df_combined = df_combined.drop(columns=['방향'])

# 결과 확인
print(df_combined)

KeyError: "['방향'] not found in axis"

In [ ]:
df_combined[df_combined['day']=='토']

,day,datetime,gu,traffic
12672,토,2018-01-06 00:00:00,강남구,15425.0
12673,토,2018-01-06 00:00:00,강동구,1447.0
12674,토,2018-01-06 00:00:00,강서구,12584.0
12675,토,2018-01-06 00:00:00,관악구,5388.0
12676,토,2018-01-06 00:00:00,광진구,6159.0
...,...,...,...,...
14971,토,2018-01-27 23:00:00,용산구,9212.0
14972,토,2018-01-27 23:00:00,은평구,5809.0
14973,토,2018-01-27 23:00:00,종로구,10465.0
14974,토,2018-01-27 23:00:00,중구,15247.0
